# Networks from OpenStreetMap


- crée un network à partir d'un fichier .osm et des trajectoires de véhiculess
- ajoute un flux de voiture sur les routes
- personnalise un Environnement pour le RL
- integrate it and run it in Flow.


In [1]:
from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams
from flow.controllers import RLController, IDMController
from flow.networks import Network
from flow.core.params import InFlows

## Crée le network

Specifie les noms des routes du network dont les vehicules peuvent être s'insérer

In [2]:
EDGES_DISTRIBUTION = [
    "-100822066",
    "4794817",
    "4783299#0",
    "155558218",
]

créer la classe Network pour spécifier les itinéraires possibles

In [3]:
class IssyOSMNetwork(Network):

    def specify_routes(self, net_params):
        return {
            "-100822066": [ #N
                "-100822066",
                "-352962858#1",
                "-352962858#0",
                "-4786940#1",
                "-4786940#0",
            ],
            
            "4794817" : [ #Loop
                "4794817",
                "4786972#0",
                "4786972#1",
                "4786972#2",
                "4786965#1",
                "4786965#2",
                "4786965#3",
                "4795729",
                "-352962858#1",
                "4795742#0",
                "4795742#1",
                "4786965#3",
                "4786965#4",
                "4786965#5",
            ],
            
            "4783299#0": [    #E
                "4783299#0",
                "4783299#1",
                "4783299#2",
                "4783299#3",
                "4783299#4",
                "4783299#5",
                "4783299#6",
                "4786940#0",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],
            
            "155558218": [
                "155558218",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],     
        }

## Ajoute les flux de voiture

`IDMController` : The Intelligent Driver Model is a car-following model specifying vehicle dynamics by a differential equation for acceleration $\dot{v}$.

`RLController` : a trainable autuonomous vehicle whose actions are dictated by an RL agent. 

In [4]:
import MinicityRouter
from flow.controllers import ContinuousRouter

In [5]:
vehicles = VehicleParams()
vehicles.add("human",acceleration_controller=(IDMController, {}),num_vehicles=5)
vehicles.add("rl",acceleration_controller=(IDMController, {}),num_vehicles=5,color="blue")

- `vehs_per_hour`: nombre de vehicule par heure, uniformément espacés. Par exemple, comme il y a $60 \times 60 = 3600$ secondes dans une heure, le parametre $\frac{3600}{5}=720$ va faire rentrer des vehicules dans le network toutes les $5$ secondes.

- `probability`: c'est la probabilité qu'un véhicule entre dans le network toutes les secondes. Par exemple, si on la fixe à $0.2$, alors chaque seconde de la simulation un véhicule aura $\frac{1}{5}$ chance d'entrer dans le network

- `period`: C'est le temps en secondes entre 2 véhicules qui sont insérés. Par exemple, le fixer à $5$ ferait rentrer des véhicules dans le network toutes les $5$ secondes (ce qui équivaut à mettre `vehs_per_hour` à $720$).

<font color='red'>
$\rightarrow$ Exactement 1 seul de ces 3 paramètres doit être configurer !
</font>

In [6]:
inflow = InFlows()
 
for edge in EDGES_DISTRIBUTION:
    inflow.add(veh_type = "human", edge = edge, probability = 0.1, depart_speed = 7)

## Personnalise un Environnement pour le RL

plus de méthodes sur : http://berkeleyflow.readthedocs.io/en/latest/

In [7]:
# import the base environment class
from flow.envs import Env
from gym.spaces.box import Box
from gym.spaces import Tuple
import numpy as np

### definition de la classe environnement 

In [8]:
class myEnv(Env):
    pass

### fonction action_space
2 actions possibles pour chaque véhicule RL : +1 acceleration ou -1 acceleration

In [9]:
class myEnv(myEnv): # update my environment class

    @property
    def action_space(self):
        num_actions = self.initial_vehicles.num_rl_vehicles
        accel_ub = self.env_params.additional_params["max_accel"]
        accel_lb = - abs(self.env_params.additional_params["max_decel"])

        return Box(low=accel_lb,
                   high=accel_ub,
                   shape=(num_actions,))

### observation_space
2 valeurs observé pour chaque véhicule: sa **position** et sa **vitesse**. En conséquence, nous avons besoin d'un espace d'observation qui est *deux fois plus grand que le nombre de véhicules* dans le network.

In [10]:
class myEnv(myEnv):  # update my environment class

    @property
    def observation_space(self):
        return Box(
            low=0,
            high=float("inf"),
            shape=(2*self.initial_vehicles.num_vehicles,))

### apply_rl_actions
`apply_rl_actions` : transforme les commandes de l'agent RL en actions réelles du simulateur.  

Pour notre exemple, l'agent RL peut spécifier que les accélérations des véhicules RL avec la fonction **apply_acceleration**

In [11]:
class myEnv(myEnv):  # update my environment class

    def _apply_rl_actions(self, rl_actions):
        # the names of all autonomous (RL) vehicles in the network
        rl_ids = self.k.vehicle.get_rl_ids()

        # use the base environment method to convert actions into accelerations for the rl vehicles
        self.k.vehicle.apply_acceleration(rl_ids, rl_actions)

### get_state

`get_state` : extrait des features de l'environnement et fournit ensuite des entrées à la stratégie fournie par l'agent RL. 

In [12]:
class myEnv(myEnv):  # update my environment class

    def get_state(self, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()

        # we use the get_absolute_position method to get the positions of all vehicles
        pos = [self.k.vehicle.get_x_by_id(veh_id) for veh_id in ids]

        # we use the get_speed method to get the velocities of all vehicles
        vel = [self.k.vehicle.get_speed(veh_id) for veh_id in ids]

        # the speeds and positions are concatenated to produce the state
        return np.concatenate((pos, vel))

### compute_reward

`compute_reward` : renvoie la récompense associée à un état donné. 

Ici, la fonction de récompense est la **vitesse moyenne de tous les véhicules actuellement sur le réseau**.

In [13]:
class myEnv(myEnv):  # update my environment class

    def compute_reward(self, rl_actions, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()

        # we next get a list of the speeds of all vehicles in the network
        speeds = self.k.vehicle.get_speed(ids)

        # finally, we return the average of all these speeds as the reward
        return np.mean(speeds)

## Lance une simulation

In [14]:
HORIZON = 2000

In [15]:
from flow.core.experiment import Experiment
from myenv import myEnv
ADDITIONAL_ENV_PARAMS = {"max_accel": 2, "max_decel": 2}

# SUMO PARAM
sumo_params = SumoParams(sim_step=0.1, render=False)

# ENVIRONMENT PARAM
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=HORIZON)

# NETWORK PARAM
net_params = NetParams(inflows=inflow, osm_path='issy.osm')

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION, spacing="random")

# dictionnaire FLOW_PARAM
flow_params = dict( exp_tag  = 'ISSY_test',
                    env_name = myEnv,  # using my new environment for the simulation
                    network  = network,
                    simulator= 'traci',
                    sim      = sumo_params,
                    env      = env_params,
                    net      = net_params,
                    veh      = vehicles,
                    initial  = initial_config)

# create EXPERIMENT with class created
exp = Experiment(flow_params)

# RUN SIMULATION SUMO
_ = exp.run(1)


**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************
4794817


AttributeError: 'NoneType' object has no attribute 'group'

In [21]:
import re
edge = '4794817'
pattern   = re.compile(r"[0-9]+")
edge_type = pattern.match(edge).group()

In [19]:
pattern.match(edge)

In [20]:
pattern

re.compile(r'[a-zA-Z]+', re.UNICODE)